## Setup Telegram bot to receive notifications

*Taken from [How to send notifications to Telegram with Python by Andrei Kushniarou](https://andrewkushnerov.medium.com/how-to-send-notifications-to-telegram-with-python-9ea9b8657bfb)*

### Create a bot

To create your new bot, you need to find a special bot called BotFather on Telegram. BotFather is the bot is used to manage all your bots. Search for it (type “BotFather” in the search bar), then type `/newbot`.

Next you will be asked to provide a user friendly "name" and unique "username" (A-Z, a-z, 0-9, _ and - are allowed). If the username is available, you'll receive an API key.

Using this API key, you can interact with the bot and send messages. 

<u>Store the API key in the .env file of this repository.</u>

### Add bot to group chat

Your newly created bot cannot just send messages to anyone or any group. To enable it to send messages, you need to add it to a Telegram group. You can create a new group and add your Telegram bot to it.

Click on “Create a New Group”, set group name of your choice and add the bot as a group member by searching for the unique username you've used for it.

The next step is to find the group’s Chat ID, which is necessary to know where to send messages. The way to find it is by sending a message to the bot and then using a small Python script to retrieve the latest messages. 

<u>Send a message in the group chat addressed directly to the bot (in private mode, by default, the bot can only read such messages).</u>

```
/hello @your_bots_username
```

### Retrieving Chat ID

Make sure you have the Python Telegram bot package installed.

1. ```pip install python-telegram-bot```
2. ```conda install conda-forge::python-telegram-bot```

In [12]:
import dotenv, os
from telegram import Bot 

#Load the token from .env file
dotenv.load_dotenv()
TELEGRAM_BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")

#Create bot object
bot = Bot(token=TELEGRAM_BOT_TOKEN)

#Get updates
updates = await bot.get_updates()

if not updates:
    print("No updates found")
else:
    for update in updates:
        if update.message:

            #Show chat id, title and message
            chat_id = update.message.chat.id
            chat_title = update.message.chat.title
            message_text = update.message.text
            print(f"Chat ID: {chat_id}  | Chat Title: {chat_title} | Message: {message_text}")


Chat ID: -4545482209  | Chat Title: ÖBB Ticket Updates | Message: /hello @oebb_ticket_update_bot


### Sending test message

When we run the cell above, we get the Chat ID of our group. The negative group identifier in Telegram is normal, don't worry.

We can send a message to the group using `bot.send_message`. After running the cell below, you should receive a message from the bot in the Chat Group you've created.

In [13]:
chat_id = -4545482209
message = "Hello world!"

await bot.send_message(text=message, chat_id=chat_id)

Message(channel_chat_created=False, chat=Chat(api_kwargs={'all_members_are_administrators': True}, id=-4545482209, title='ÖBB Ticket Updates', type=<ChatType.GROUP>), date=datetime.datetime(2024, 10, 12, 18, 26, 34, tzinfo=datetime.timezone.utc), delete_chat_photo=False, from_user=User(first_name='ÖBB-TicketBot', id=7433926112, is_bot=True, username='oebb_ticket_update_bot'), group_chat_created=False, message_id=4, supergroup_chat_created=False, text='Hello world!')

## Scraping Nightjet booking page data

Booking tickets for Nightjet train is possible on [https://www.nightjet.com/en/ticket-buchen](https://www.nightjet.com/en/ticket-buchen#/home). We are interested in the maximal available date in the calendar widget on this page.

Basically we have to options:
- *high-level and hard approach:* automate a real browser using `selenium` or, in other words, make the browser repeat all the user actions needed to open the calendar, navigate to correct month and inspect the DOM to find latest bookable date

- *low-level and easy if possible approach:* investigate what is happening under the hood - explore the "Network"  being made by the webpage. Hopefully we can find how is the data for the maximal enabled date in calendar fetched by the website and then repeat that request ourselves.

### Network investigation

In Chrome you can open developer tools with *Ctrl+Shift+I*. After checking the Network traffic upon page load, I found a JSON object fetched from `https://www.nightjet.com/nj-booking-ocp/init/start`.

Object contains a lot of data, including some local translation dictionaries, but all the way at the end you can find an interesting property - `maxBookableDate`.

### Putting it all together

With the bot setup and a source of data identified, we just need to put everything together.

The request for `start` JSON is a POST request, and needs to have `{"lang": "en"}` in the body. 
The date comes in the `YYYY-mm-dd` format. We just need to parse it into Python `datetime` to be able to compare it with our desired date.

After that, we just have to decide what messages do we sent to our Telegram based on the result.



In [16]:
import requests
from datetime import datetime

#Desired booking date
booking_date = datetime(2024, 12, 20)

#Fetch data used to populate the Nightjet booking webapp
page_data = requests.post("https://www.nightjet.com/nj-booking-ocp/init/start", json={"lang": "en"})
if page_data.ok:
    #Parse maximal possible booking date from the data
    max_bookable_date = datetime.strptime(page_data.json()["maxBookableDate"], "%Y-%m-%d")

    if max_bookable_date >= booking_date:
        message = f"🔥🚨 Tickets are available for {booking_date.date()}! 🚨🔥"
    else:
        message = f"⏳ No luck with booking for {booking_date.date()} yet."""
else:
    message = "😢 Couldn't get required data from Nightjet booking website."


await bot.send_message(text=message, chat_id=chat_id)


Message(channel_chat_created=False, chat=Chat(api_kwargs={'all_members_are_administrators': True}, id=-4545482209, title='ÖBB Ticket Updates', type=<ChatType.GROUP>), date=datetime.datetime(2024, 10, 12, 19, 21, 38, tzinfo=datetime.timezone.utc), delete_chat_photo=False, from_user=User(first_name='ÖBB-TicketBot', id=7433926112, is_bot=True, username='oebb_ticket_update_bot'), group_chat_created=False, message_id=5, supergroup_chat_created=False, text='⏳ No luck with booking for 2024-12-20 yet.')